<a href="https://colab.research.google.com/github/ridhima31/VIZ-Test/blob/master/Transpose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PySpark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=2ef16fdecd656ce6db52a79cf9831b5b5c7d8793e3267241e28cdfcb0d86bb25
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built PySpark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

In [13]:
# Create a Spark session
spark = SparkSession.builder.appName("TransposeExample").getOrCreate()

# Sample DataFrame
data = [("A", 1, 4, 8), ("A", 1, 5,9), ("C", 3, 6,1)]
columns = ["ID", "Value1", "Value2"]


In [14]:
df = spark.createDataFrame(data, columns)

In [15]:
print("Original DataFrame:")
df.show()

Original DataFrame:
+---+------+------+---+
| ID|Value1|Value2| _4|
+---+------+------+---+
|  A|     1|     4|  8|
|  A|     1|     5|  9|
|  C|     3|     6|  1|
+---+------+------+---+



In [17]:
transposed_df = df.groupBy("ID").pivot("Value1").agg(expr("last(Value2)"))

print("\nTransposed DataFrame:")
transposed_df.show()


Transposed DataFrame:
+---+----+----+
| ID|   1|   3|
+---+----+----+
|  C|NULL|   6|
|  A|   5|NULL|
+---+----+----+



In [22]:
from pyspark.sql.functions import col, expr, explode

### Explode column to rows

In [29]:
data = [("A", 1, 4,5), ("B", 2, 5,6), ("C", 3, 6,7)]
columns = ["ID", "Value1", "Value2","Value3"]

In [30]:
df = spark.createDataFrame(data, columns)

In [20]:
print("Original DataFrame:")
df.show()

Original DataFrame:
+---+------+------+
| ID|Value1|Value2|
+---+------+------+
|  A|     1|     4|
|  B|     2|     5|
|  C|     3|     6|
+---+------+------+



In [35]:
transposed_df = (
    df.selectExpr("ID", "stack(2, 'Value1', Value1, 'Value2', Value2) as (Column, TransposedValue)","Value3")
)

print("\nTransposed DataFrame using explode:")
transposed_df.show()


Transposed DataFrame using explode:
+---+------+---------------+------+
| ID|Column|TransposedValue|Value3|
+---+------+---------------+------+
|  A|Value1|              1|     5|
|  A|Value2|              4|     5|
|  B|Value1|              2|     6|
|  B|Value2|              5|     6|
|  C|Value1|              3|     7|
|  C|Value2|              6|     7|
+---+------+---------------+------+

